In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

from bs4 import BeautifulSoup

import time
import os

import pandas as pd
from tqdm import tqdm

In [ ]:
company_list = pd.read_excel("../iss_最終整理名單.xls")

In [ ]:
company_list.head()

In [ ]:
transcripts_Summary_href = "https://www.capitaliq.com/CIQDotNet/Transcripts/Summary.aspx"

In [ ]:
def getBrowser(username=None, password=None):
    # start web browser
    browser=webdriver.Chrome(executable_path="./chromedriver")
    browser.get(transcripts_Summary_href)
    
    try:
        myElem = WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.ID, 'username')))
    except TimeoutException:
        print(f"Loading search button took too much time!")
    
    browser.find_element_by_xpath('//*[@id="username"]').send_keys(username)
    browser.find_element_by_xpath('//*[@id="password"]').send_keys(password)
    browser.find_element_by_xpath('//*[@id="myLoginButton"]').click()
    
    return browser

In [ ]:
def getHTML(browser, startID=0, endID=company_list.shape[0], IDs = []):

    first_half_XPath = '//*[@id="_transcriptsGrid__dataGrid"]/tbody/tr['
    last_half_XPath = ']/td[3]/span/a'
    
    try:
        company_completed = os.listdir("transcripts")
        startID = int(max(company_completed)) - 1
        # startID -= 1 to reprocess the last company for safety 
    except Exception as e: 
        print(f"Get Start ID error :\n{e}")
    
    print(f"Starting from ID : {startID}")
    
    #for i in IDs:  # 自己選擇要爬哪些ID
    for i in tqdm(range(startID, endID)):  # 正常流程照順序爬

        if not company_list["transcript"][i]:
            continue

        # Get the IQID and ID of current company
        #
        # e.g. IQID of company "3Com Corporation" is IQ24107
        #      ID   of company "3Com Corporation" is 3
        #
        company_IQID = company_list["Excel Company ID"][i][2:]
        company_ID = company_list["id"][i]

        # Get the href of summary of all transcripts of current company
        #
        # e.g. https://www.capitaliq.com/CIQDotNet/Transcripts/Summary.aspx?companyId=24107
        #
        company_href = transcripts_Summary_href + "?companyId=" + str(company_IQID)

        # Go to that href
        browser.get(company_href)
        time.sleep(10)
        
        # Recheck if this company has no transcript
        table_length = len(browser.find_elements_by_xpath('//*[@id="_transcriptsGrid__dataGrid"]/tbody/tr'))    
        if table_length == 1:
            continue

        # Get the total number of transcript 
        #
        # e.g. Extract "14" from "Viewing 1-14 of 14"
        #
        all_trans_numbers = None
        try:
            all_trans_numbers_XPath = first_half_XPath + str(table_length) + ']/td'
            all_trans_numbers = browser.find_element_by_xpath(all_trans_numbers_XPath).text
            all_trans_numbers = int(all_trans_numbers.split()[3])
        except:
            print(company_ID, "found no number")
            continue

        # Click 'View All' botton if it exist
        try:
            browser.find_element_by_link_text('View All').click()

            # 有時候 View All 還沒跑完就開始拉資料了，所以等一下，
            # 因為 transcripts 越多就要等越久，
            # 所以這裡我設需要等 (all_trans_numbers / 10) 秒
            time.sleep(all_trans_numbers / 10)
        except:
            pass


        # Get all of the hrefs and names of transcipts of company
        all_transcipts_href = []
        all_transcipts_name = []
        table_length = len(browser.find_elements_by_xpath('//*[@id="_transcriptsGrid__dataGrid"]/tbody/tr'))
        for j in range(2, table_length):

            full_XPath = first_half_XPath + str(j) + last_half_XPath

            try:
                page_href = browser.find_element_by_xpath(full_XPath).get_attribute('href')
                all_transcipts_href.append(page_href)

                name = browser.find_element_by_xpath(full_XPath).text
                all_transcipts_name.append(name)
            except:
                pass

        #print(all_trans_numbers, len(all_transcipts_href))

        # Check for missing transcipts 
        # 因為有些 transcipts 還沒被放上去只有文字標題，
        # 但是它還是被算在 all_trans_numbers 裡面，
        # 所以選擇 >10 確保沒有遺失 transcipts
        if all_trans_numbers - len(all_transcipts_href) > 10:
            print("not retrieve all trans in id :", company_ID)

        filepath = os.path.join('transcripts', str(company_ID))
        # Check if the folder exists. If not, create one
        if not os.path.exists(filepath):
            os.makedirs(filepath)

        for j in range(len(all_transcipts_href)):

            browser.get(all_transcipts_href[j])
            time.sleep(0.125)
            html = browser.page_source

            filename = os.path.join(filepath, all_transcipts_name[j].replace('/', '_')) + '.html'
            #print(filename)
            try:
                f = open(filename, "w")
                f.write(str(BeautifulSoup(html, 'html.parser').prettify()))
                f.close()
            except:
                pass

        if len(all_transcipts_href) != len(os.listdir(filepath)):
            print(len(all_transcipts_href) - len(os.listdir(filepath)), \
                    "transcript missed for id :", company_ID)

In [ ]:
browser = getBrowser(username = "", password = "")

# Waiting for login
time.sleep(10)

getHTML(browser)